In [1]:
# подгрузим модули
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


In [2]:
# загрузим данные
dataset = pd.read_csv('Market_Basket_Optimisation.csv', header = None)

In [3]:
# посомтрим на датасет
dataset.head()


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,shrimp,almonds,avocado,vegetables mix,green grapes,whole weat flour,yams,cottage cheese,energy drink,tomato juice,low fat yogurt,green tea,honey,salad,mineral water,salmon,antioxydant juice,frozen smoothie,spinach,olive oil
1,burgers,meatballs,eggs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,chutney,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,turkey,avocado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,mineral water,milk,energy bar,whole wheat rice,green tea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
dataset.fillna(method = 'ffill',axis = 1, inplace = True)

In [5]:
dataset.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,shrimp,almonds,avocado,vegetables mix,green grapes,whole weat flour,yams,cottage cheese,energy drink,tomato juice,low fat yogurt,green tea,honey,salad,mineral water,salmon,antioxydant juice,frozen smoothie,spinach,olive oil
1,burgers,meatballs,eggs,eggs,eggs,eggs,eggs,eggs,eggs,eggs,eggs,eggs,eggs,eggs,eggs,eggs,eggs,eggs,eggs,eggs
2,chutney,chutney,chutney,chutney,chutney,chutney,chutney,chutney,chutney,chutney,chutney,chutney,chutney,chutney,chutney,chutney,chutney,chutney,chutney,chutney
3,turkey,avocado,avocado,avocado,avocado,avocado,avocado,avocado,avocado,avocado,avocado,avocado,avocado,avocado,avocado,avocado,avocado,avocado,avocado,avocado
4,mineral water,milk,energy bar,whole wheat rice,green tea,green tea,green tea,green tea,green tea,green tea,green tea,green tea,green tea,green tea,green tea,green tea,green tea,green tea,green tea,green tea


In [6]:
# dataset.columns
# dataset.shape
# dataset.tail()
# dataset.info()
# dataset.describe()

In [92]:
#создаим матрицу 
transactions = []
for i in range(0, 7501): 
    transactions.append([str(dataset.values[i,j]) for j in range(0, 20)])
#print(transactions)

In [8]:
import apyori
from apyori import apriori

In [13]:
%%time
# и обучимся правилам. Обратите внимание, что пороговые значения мы вибираем сами в зависимости от того, /
# насколкьо "сильные" правила мы хотим получить
# min_support -- минимальный support для правил (dtype = float).
# min_confidence -- минимальное значение confidence для правил (dtype = float)
# min_lift -- минимальный lift (dtype = float)
# max_length -- максимальная длина itemset (вспоминаем про k-itemset)  (dtype = integer)

result = list(apriori(transactions, min_support = 0.003, min_confidence = 0.3, min_lift = 4,  min_length = 2))

# print(result)

Wall time: 838 ms


In [14]:
import shutil, os 
import json #преобразовывать будем в json, используя встроенные в модуль методы
try:
    from StringIO import StringIO
except ImportError:
    from io import StringIO
    
    output = []
for RelationRecord in result:
    o = StringIO()
    apyori.dump_as_json(RelationRecord, o)
    output.append(json.loads(o.getvalue()))
data_df = pd.DataFrame(output)

# и взгялнем на итоги
pd.set_option('display.max_colwidth', -1)

from IPython.display import display, HTML

display(HTML(data_df.to_html()))

#support - частотность itemsseta во всех транзакциях
#confidence - достоверность, как часто оно срабатыает для датасета
#lift - насколько зависмы товары


,items,ordered_statistics,support
0,"[escalope, pasta]","[{'items_base': ['pasta'], 'items_add': ['escalope'], 'confidence': 0.3728813559322034, 'lift': 4.700811850163794}]",0.005866
1,"[pasta, shrimp]","[{'items_base': ['pasta'], 'items_add': ['shrimp'], 'confidence': 0.3220338983050847, 'lift': 4.506672147735896}]",0.005066
2,"[cereals, ground beef, spaghetti]","[{'items_base': ['cereals', 'spaghetti'], 'items_add': ['ground beef'], 'confidence': 0.45999999999999996, 'lift': 4.681763907734057}]",0.003066
3,"[chocolate, ground beef, herb & pepper]","[{'items_base': ['chocolate', 'herb & pepper'], 'items_add': ['ground beef'], 'confidence': 0.4411764705882354, 'lift': 4.4901827759597746}]",0.003999
4,"[french fries, ground beef, herb & pepper]","[{'items_base': ['french fries', 'herb & pepper'], 'items_add': ['ground beef'], 'confidence': 0.46153846153846156, 'lift': 4.697421981004071}]",0.003200
5,"[ground beef, herb & pepper, spaghetti]","[{'items_base': ['herb & pepper', 'spaghetti'], 'items_add': ['ground beef'], 'confidence': 0.3934426229508197, 'lift': 4.004359721511667}]",0.006399
6,"[ground beef, spaghetti, tomato sauce]","[{'items_base': ['spaghetti', 'tomato sauce'], 'items_add': ['ground beef'], 'confidence': 0.4893617021276596, 'lift': 4.980599901844742}]",0.003066
7,"[mineral water, olive oil, whole wheat pasta]","[{'items_base': ['mineral water', 'whole wheat pasta'], 'items_add': ['olive oil'], 'confidence': 0.4027777777777778, 'lift': 6.115862573099416}]",0.003866
8,"[chocolate, frozen vegetables, mineral water, shrimp]","[{'items_base': ['chocolate', 'frozen vegetables', 'mineral water'], 'items_add': ['shrimp'], 'confidence': 0.32876712328767127, 'lift': 4.600899611531385}, {'items_base': ['chocolate', 'mineral water', 'shrimp'], 'items_add': ['frozen vegetables'], 'confidence': 0.4210526315789474, 'lift': 4.417224880382776}]",0.003200
9,"[frozen vegetables, milk, mineral water, olive oil]","[{'items_base': ['frozen vegetables', 'milk', 'mineral water'], 'items_add': ['olive oil'], 'confidence': 0.30120481927710846, 'lift': 4.573557387444516}, {'items_base': ['milk', 'mineral water', 'olive oil'], 'items_add': ['frozen vegetables'], 'confidence': 0.39062500000000006, 'lift': 4.098011363636364}]",0.003333


In [61]:
import numpy as np
"""
Класс инициируется 3мя параметрами:
- min_supp - минимальный support  который мы рассматриваем для ItemSet. Рассчитывается как % от количества транзакций
- max_items - максимальное количество елементов в нашем ItemSet
- min_items - минимальное количество элементов ItemSet
"""

class Eclat:
    #инициализация объекта класса
    def __init__(self, min_support = 0.01, max_items = 5, min_items = 2):
        self.min_support = min_support
        self.max_items = max_items
        self.min_items = min_items
        self.item_lst = list()
        self.item_len = 0
        self.item_dict = dict()
        self.final_dict = dict()
        self.data_size = 0
    
    #создание словаря из ненулевых объектов из всех транзакций (вертикальный датасет)
    def read_data(self, dataset):
        for index, row in dataset.iterrows():
            row_wo_na = row.dropna().unique()
            for item in row_wo_na:
                item = item.strip()
                if item in self.item_dict:
                    self.item_dict[item][0] += 1
                else:
                    self.item_dict.setdefault(item, []).append(1)
                self.item_dict[item].append(index)
        #задаем переменные экземпляра (instance variables)
        self.data_size = dataset.shape[0]
        self.item_lst = list(self.item_dict.keys())
        self.item_len = len(self.item_lst)
        self.min_support = self.min_support * self.data_size
        #print ("min_supp", self.min_support)
        
    #рекурсивный метод для поиска всех ItemSet по алгоритму Eclat
    #структура данных: {Item: [Supp number, tid1, tid2, tid3, ...]}
    def recur_eclat(self, item_name, tids_array, minsupp, num_items, k_start):
        if tids_array[0] >= minsupp and num_items <= self.max_items:
            for k in range(k_start+1, self.item_len):
                if self.item_dict[self.item_lst[k]][0] >= minsupp:
                    new_item = item_name + " | " + self.item_lst[k]
                    new_tids = np.intersect1d(tids_array[1:], self.item_dict[self.item_lst[k]][1:])
                    new_tids_size = new_tids.size
                    new_tids = np.insert(new_tids, 0, new_tids_size)
                    if new_tids_size >= minsupp:
                        if num_items >= self.min_items: self.final_dict.update({new_item: new_tids})
                        self.recur_eclat(new_item, new_tids, minsupp, num_items+1, k)
    
    #последовательный вызов функций определенных выше
    def fit(self, dataset):
        i = 0
        self.read_data(dataset)
        for w in self.item_lst:
            self.recur_eclat(w, self.item_dict[w], self.min_support, self.min_items, i)
            i+=1
        return self
        
    #вывод в форме словаря {ItemSet: support(ItemSet)}
    def transform(self):
        return {k: "{0:.4f}%".format((v[0]+0.0)/self.data_size*100) for k, v in self.final_dict.items()}

In [66]:
%%time
#создадим экземпляр класса с нужными нам параметрами
model = Eclat(min_support = 0.01, max_items = 19, min_items = 2)

#обучим
model.fit(dataset)

Wall time: 6.38 s


In [67]:
#и визуализируем результаты
model.transform()

{'shrimp | green tea': '1.1465%',
 'shrimp | mineral water': '2.3597%',
 'shrimp | eggs': '1.4131%',
 'shrimp | milk': '1.7598%',
 'shrimp | frozen vegetables': '1.6664%',
 'shrimp | spaghetti': '2.1197%',
 'shrimp | chocolate': '1.7998%',
 'shrimp | tomatoes': '1.1199%',
 'shrimp | pancakes': '1.0532%',
 'shrimp | ground beef': '1.1465%',
 'avocado | mineral water': '1.1598%',
 'low fat yogurt | mineral water': '2.3997%',
 'low fat yogurt | eggs': '1.6798%',
 'low fat yogurt | milk': '1.3198%',
 'low fat yogurt | french fries': '1.3332%',
 'low fat yogurt | frozen vegetables': '1.0132%',
 'low fat yogurt | spaghetti': '1.5198%',
 'low fat yogurt | chocolate': '1.4798%',
 'green tea | mineral water': '3.1063%',
 'green tea | frozen smoothie': '1.1199%',
 'green tea | burgers': '1.7464%',
 'green tea | eggs': '2.5463%',
 'green tea | turkey': '1.1865%',
 'green tea | milk': '1.7598%',
 'green tea | french fries': '2.8530%',
 'green tea | frozen vegetables': '1.4398%',
 'green tea | spag

In [68]:
pip install pyfpgrowth

Note: you may need to restart the kernel to use updated packages.


In [69]:
import pyfpgrowth

In [84]:
patterns = pyfpgrowth.find_frequent_patterns(dataset.values, 2) 
rules = pyfpgrowth.generate_association_rules(patterns, 30)

KeyboardInterrupt: 

In [85]:
#Сгенериуем паттерны
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder

te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions)
df = pd.DataFrame(te_ary, columns=te.columns_)
#df

In [86]:
%%time
#Сгенериуем правила
from mlxtend.frequent_patterns import fpgrowth

fpgrowth(df, min_support=0.03)

Wall time: 164 ms


,support,itemsets
0,0.238368,(72)
1,0.132116,(54)
2,0.076523,(65)
3,0.071457,(97)
4,0.065858,(81)
5,0.063325,(48)
6,0.047460,(60)
7,0.042528,(92)
8,0.033329,(4)
9,0.031862,(33)


In [90]:
fpgrowth(df, min_support=0.04, use_colnames=True)

,support,itemsets
0,0.238368,(mineral water)
1,0.132116,(green tea)
2,0.076523,(low fat yogurt)
3,0.071457,(shrimp)
4,0.065858,(olive oil)
5,0.063325,(frozen smoothie)
6,0.047460,(honey)
7,0.042528,(salmon)
8,0.179709,(eggs)
9,0.087188,(burgers)
